# Machine Learning con Python

Carichiamo i dati del famaso dataset iris.

Citazione
Fisher,R. A.. (1988). Iris. UCI Machine Learning Repository. https://doi.org/10.24432/C56C76. This dataset is licensed under a Creative Commons Attribution 4.0 International (CC BY 4.0) license. https://creativecommons.org/licenses/by/4.0/legalcode

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
iris=sns.load_dataset("iris")

In [ ]:
iris.head(5)

In [ ]:
#se non hai installato seaborn
iris = pd.read_csv(filepath_or_buffer=r"C:\Users\ianto\Desktop\Corso Python\file\iris.csv",
                  header=1)

Diamo preliminarmente un'occhiata ai dati

In [ ]:
iris.info()

Partiamo con una lista contenente i valori di un nuovo iris

In [ ]:
nuovo_iris = [4.3, 3.2, 1.4, 0.3]

In [ ]:
#Osservazione: su python posso valorizzare più variabili contemporneamente con questa sintassi
#x, y, z, w = 4.3, 3.2, 1.4, 0.3

Creiamo una colonna distanza

In [ ]:
iris["distanza"] = np.sqrt( (iris["sepal_length"]-nuovo_iris[0])**2 + \
                            (iris["sepal_width"]-nuovo_iris[1])**2 + \
                            (iris["petal_length"]-nuovo_iris[2])**2 + \
                            (iris["petal_width"]-nuovo_iris[3])**2) 

Ordiniam i valori dello storico per distanza e prendiamo l'iris più vicino

In [ ]:
iris.sort_values(by="distanza", ascending = True).head(1)

Cerhiamo di generalizzare la procedura per avere dei dati sull'accuratezza aspettata.

Divido il dataset in un 70% di training e un 30% di test

In [ ]:
training = iris.sample(frac=0.7, random_state=0)

In [ ]:
training.head(5)

Per prendere il 30% restante, cancello da iris le righe con l'indice presente nel training

In [ ]:
test = iris.drop(training.index)

In [ ]:
test.head(5)

Implementiamo l'algoritmo di classificazione k-neighbours con k=1. Per prima cosa facciamo diventare l'indice di test una colonna, in modo da usarlo come identificativo.

In [ ]:
test = test.reset_index()

In [ ]:
test.head(5)

Eseguiamo una cross merge di test e training

In [ ]:
merge = pd.merge(test,
         training,
         how="cross",
         suffixes= ("_test","_training")    
      )

Guardiamo il nuovo Dataset

In [ ]:
merge.head(5)

In [ ]:
merge.tail(5)

Testiamo le dimensioni

In [ ]:
len(training),len(test),len(merge)

Creiamo una colonna che calcoli la distanza tra i dati di traning e quelli di test

In [ ]:
merge["distanza"] = pow( pow(merge["sepal_length_training"]-merge["sepal_length_test"],2)+
                         pow(merge["sepal_width_training"]-merge["sepal_width_test"],2)+
                         pow(merge["petal_length_training"]-merge["petal_length_test"],2)+
                         pow(merge["petal_width_training"]-merge["petal_width_test"],2) ,
                        0.5)

Cerchiamo l'iris del training più vicino all'iris di test con index 0

In [ ]:
merge.query("index == 0").sort_values(by="distanza",ascending=True).head(1)

Ordiniamo i dati per distanza crescente e rimuoviamo i duplicati per la colonna index mantenendo il primo elemento. In questo modo avremo per ogni riga di test la riga di training più vicina

In [ ]:
predizioni = merge.sort_values(by="distanza",ascending=True).drop_duplicates(subset=["index"], keep = "first")
predizioni.head(5)

Calcoliamo il numero di errori

In [ ]:
predizioni.query("species_test != species_training")

Calcoliamo l'accuratezza

In [ ]:
len(predizioni.query("species_test == species_training"))/len(predizioni)

# Ripetiamo il procedimento con scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import pandas as pd
import numpy as np
import seaborn as sns

Importiamo il dataset iris

In [ ]:
iris=sns.load_dataset("iris")

In [ ]:
#se non hai installato seaborn
iris = pd.read_csv(filepath_or_buffer=r"C:\Users\ianto\Desktop\Corso Python\file\iris.csv",
                  header=1)

Ripassiamo come si "elimina" una colonna

In [ ]:
iris.drop("species", axis=1).head(5)

Dividiamo i dati in training e test utilizzando from il metodo train_test_split di scikit-learn

In [ ]:
features_training,features_test, target_training, target_test = train_test_split(iris.drop("species", axis=1),
                                                                   iris[["species"]],
                                                                   test_size = 0.3,
                                                                   shuffle = True,
                                                                   random_state = 1)

Guardiamo i quattro dataset

In [ ]:
features_training.head(5)

In [ ]:
target_training.head(5)

In [ ]:
features_test.head(5)

In [ ]:
target_test.head(5)

Istanziamo l'algoritmo K-Neighbours tramite scikit-learn

In [ ]:
knc = KNeighborsClassifier(n_neighbors=1)

Alleniamolo soltanto sui dati di training tramite il metodo fit

In [ ]:
#per eliminare il warning usare target_training.values.ravel()
knc.fit(features_training,target_training)

Utilizziamo il metodo allenato per calcolare le predizioni sui dati di test tramite il metodo predict

In [ ]:
predizioni = knc.predict(features_test)
predizioni

Con il metodo score posso calcolare quante di queste predizioni sono corrette

In [ ]:
score = knc.score(features_test,target_test)
score

Ricreiamo il dataset features_test aggiungendo i valori delle classi reali e predette

In [ ]:
iris_completo = features_test.copy()
iris_completo["species_originale"]=target_test["species"]
iris_completo["species_predetta"]=predizioni
iris_completo.head(5)

Vediamo gli errori

In [ ]:
iris_completo.query("species_originale != species_predetta")

Calcoliamo di nuovo l'accuratezza "manualmente"

In [ ]:
len(iris_completo.query("species_originale == species_predetta"))/len(iris_completo)

Calcoliamo la matrice di confusione

In [ ]:
confusion_matrix(target_test, predizioni)

Graficamente

In [ ]:
ConfusionMatrixDisplay.from_predictions(target_test, predizioni)

## Utilizziamo altri algoritmi di Machine Learning ed estendiamo il processo di lavorazione dei dati

In [ ]:
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import pandas as pd
import numpy as np
import seaborn as sns

Reimportiamo il dataset e dividiamo i dati in training e test (random_state = 0 ci garantisce di avere gli stessi dati di prima)

In [ ]:
iris=sns.load_dataset("iris")

In [ ]:
features_training,features_test, target_training, target_test = train_test_split(iris.drop("species", axis=1),
                                                                   iris[["species"]],
                                                                   test_size = 0.3,
                                                                   shuffle = True,
                                                                   random_state = 1)

Valorizziamo i valori mancanti. Partiamo istanziando la classe SimpleImputer di Scikit-learn

In [ ]:
imputer = SimpleImputer(strategy='mean')

Alleniamo l'oggetto imputer sui dati di training

In [ ]:
imputer.fit(features_training)

Utilizziamo il metodo allenato per trasformare i dati di training e di test

In [ ]:
features_training_imputed = pd.DataFrame(
    imputer.transform(features_training),
    columns=features_training.columns,
    index=features_training.index
)

In [ ]:
features_training_imputed.head(5)

In [ ]:
features_test_imputed = pd.DataFrame(
    imputer.transform(features_test),
    columns=features_test.columns,
    index=features_test.index
)

In [ ]:
features_test_imputed.head(5)

Applichiamo la normalizzazione dei dati. Partiamo istanziando la classe StandardScaler di Scikit-learn

In [ ]:
scaler = StandardScaler() 

Alleniamo l'oggetto sui dati di training

In [ ]:
scaler.fit(features_training_imputed)

Utilizziamo il metodo allenato per trasformare i dati di training e di test

In [ ]:
features_training_standard = pd.DataFrame(
    scaler.transform(features_training_imputed),
    columns=features_training_imputed.columns,
    index=features_training_imputed.index
)

In [ ]:
features_training_standard.head(5)

In [ ]:
features_test_standard = pd.DataFrame(
    scaler.transform(features_test_imputed),
    columns=features_test_imputed.columns,
    index=features_test_imputed.index
)

In [ ]:
features_test_standard.head(5)

Applichiamo un altro algoritmo di Machine Learning: il Perceptron. Istanziamo la classe relativa da scikit-learn

In [ ]:
pcp = Perceptron(random_state = 0, alpha=0.001)

Alleniamola soltanto sui dati di training

In [ ]:
#per eliminare il warning usare target_training.values.ravel()
pcp.fit(features_training_standard,target_training )

Utilizziamo il metodo allenato per calcolare le predizioni sui dati di test

In [ ]:
predizioni = pcp.predict(features_test_standard)
predizioni

Con il metodo score posso calcolare quante di queste predizioni sono corrette

In [ ]:
score = pcp.score(features_test_standard,target_test)
score

Ricreiamo il dataset features_test aggiungendo i valori delle classi reali e predette

In [ ]:
iris_completo = features_test.copy()
iris_completo["species_originale"]=target_test["species"]
iris_completo["species_predetta"]=predizioni
iris_completo.head(5)

In [ ]:
iris_completo.query("species_originale != species_predetta")

Calcoliamo di nuovo l'accuratezza "manualmente"

In [ ]:
len(iris_completo.query("species_originale == species_predetta"))/len(iris_completo)

Calcoliamo la matrice di confusione

In [ ]:
confusion_matrix(target_test, predizioni)

Graficamente

In [ ]:
ConfusionMatrixDisplay.from_predictions(target_test, predizioni)

## Pipeline con scikit-learn

Invece di eseguire manualmente i singoli passi dell'algoritmo di Machine Learning, possiamo inglobarli all'interno di una pipeline di scikit-learn

In [ ]:
from sklearn.pipeline import Pipeline

Reimportiamo i dati e applichiamo la divisione in training e test

In [ ]:
iris=sns.load_dataset("iris")

In [ ]:
features_training,features_test, target_training, target_test = train_test_split(iris.drop("species", axis=1),
                                                                   iris[["species"]],
                                                                   test_size = 0.3,
                                                                   random_state = 1)

Creiamo la pipeline

In [ ]:
my_first_pipeline = Pipeline([('si',SimpleImputer(strategy='mean')),
                              ('st',StandardScaler()),
                              ('ml', Perceptron(random_state = 0))
                               ])

Alleniamo la pipeline sui dati di training

In [ ]:
#per eliminare il warning usare target_training.values.ravel()
my_first_pipeline.fit(features_training,target_training)

Calcoliamo le predizioni sui dati di test

In [ ]:
predizioni = my_first_pipeline.predict(features_test)
predizioni

Valutiamo lo score

In [ ]:
my_first_pipeline.score(features_test,target_test)

Creo il dataset completo di dati reali e predizioni

In [ ]:
iris_completo = features_test.copy()
iris_completo["species_originale"]=target_test["species"]
iris_completo["species_predetta"]=predizioni
iris_completo.head(5)

calcoliamo di nuovo manualmente lo score

In [ ]:
len(iris_completo.query("species_originale == species_predetta"))/len(iris_completo)

Creiamo una nuova pipeline con un algoritmo diverso di Machine Learning, l'albero di decisione

In [ ]:
my_second_pipeline = Pipeline([('si',SimpleImputer(strategy='mean')),
                              ('st',StandardScaler()),
                              ('ml', DecisionTreeClassifier(random_state = 0))
                               ])

Alleno la pipeline e calcolo le accuratezze

In [ ]:
my_second_pipeline.fit(features_training,target_training)

In [ ]:
predizioni = my_second_pipeline.predict(features_test)

In [ ]:
confusion_matrix(target_test, predizioni)

In [ ]:
ConfusionMatrixDisplay.from_predictions(target_test, predizioni)

In [ ]:
my_second_pipeline.score(features_test,target_test)

Posso concludere che questo secondo algoritmo performa meglio del primo? 

No, perché potrebbe dalla particolare suddivisione dei dati in training e test. Ho bisogno di un metodo di confronto più robusto

Prima un piccolo approfondimento sui decision tree

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
plot_tree(decision_tree = my_second_pipeline.steps[2][1],          
          feature_names = ["sepal_length","sepal_width","petal_length","petal_width"],
          class_names = ['iris setosa', 'iris versicolor', 'iris virginica'],
          filled = True
         );

## Convalida incrociata

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

Guardiamo i "parametri" della pipeline

In [ ]:
my_second_pipeline.get_params()

Possiamo assegnare ad alcuni di questi parametri più valori differenti.

In [ ]:
parametri_griglia = [{"st":[StandardScaler(),MinMaxScaler()],
                      "ml":[DecisionTreeClassifier(random_state=0),LogisticRegression(random_state=0),RandomForestClassifier(random_state=0)]
                        }]

Creiamo la griglia per eseguire la convalida incrociata tramite il metodo GridSearchCV di scikit-learn

In [ ]:
my_grid = GridSearchCV(estimator = my_second_pipeline,
                      param_grid = parametri_griglia,
                      cv = 5)

Alleno la griglia sui dataset di training. Eseguirò 2x2x5=20 esecuzioni, per 4 combinazioni diverse di parametri.

Fissata una combinazione di valori della griglia (ad esempio StandardScaler e DecisionTreeClassifier), eseguirò la pipeline su 5 diverse suddivisioni del dataset di training. 

Per ognuna delle 5, eseguiro il training sul 80% dei dati e valuterò l'accuratezza sul restante 20% di convalida.

In [ ]:
#per eliminare il warning usare target_training.values.ravel()
my_grid.fit(features_training,target_training.values.ravel())

Visualizziamo lo score più alto raggiunto tra le 4 combinazioni

In [ ]:
my_grid.best_score_

Vediamo la combinazione per la quale ho ottenuto i valori migliori

In [ ]:
my_grid.best_params_

Creo la pipeline associata alla migliore combinazione

In [ ]:
best_pipeline = my_grid.best_estimator_
best_pipeline

A questo punto ho selezionato il modello migliore.

Per valutare l'accuratezza, rialleno la pipeline su tutti i dati di training e poi calcolo l'accuratezza sui dati di test

In [ ]:
best_pipeline.fit(features_training,target_training)

In [ ]:
best_pipeline.score(features_test,target_test)

Gli alberi decisionali possono essere rappresentati graficamente!

In [ ]:
plot_tree(decision_tree = best_pipeline.steps[2][1],          
          feature_names = ["sepal_length","sepal_width","petal_length","petal_width"],
          class_names = ['iris setosa', 'iris versicolor', 'iris virginica'],
          filled = True
         );

In [ ]:
best_pipeline.steps[2][1]

## Regressione

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

Importiamo il dataset diamonds

In [ ]:
diamonds=sns.load_dataset("diamonds")

In [ ]:
diamonds.head(5)

features:
- carat peso del diamante
- cut è la qualità del taglio secondo questa scala Fair, Good, Very Good, Premium, Ideal
- color riguarda il colore secondo questa scala J, I, H, G, F, E, D 
- clarity la chiarezza secondo questa scala I3, I2, I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF, FL
- depth altezza in percentuale
- table larghezza in percentuale
- x lunghezza in millimetri
- y largheza in millimetri
- z prodonfita in millimetri

target:
- price

Codifichiamo le variabili non quantitative (sono comunque tutte ordinali).

In [ ]:
codifica_cut = {"Fair":1, "Good":2, "Very Good":3, "Premium":4, "Ideal":5}
codifica_color = {"J":1, "I":2, "H":3, "G":4, "F":5, "E":6, "D":7}
codifica_clarity = {"I3":1, "I2": 2, "I1":3, "SI2":4, "SI1":5, "VS2":6, "VS1":7, "VVS2":8, "VVS1":9, "IF":10, "FL":11}

In [ ]:
diamonds["cut"] = diamonds["cut"].map(codifica_cut).astype("int64")
diamonds["color"] = diamonds["color"].map(codifica_color).astype("int64")
diamonds["clarity"] = diamonds["clarity"].map(codifica_clarity).astype("int64")

Vediamo il nuovo dataset

In [ ]:
diamonds.head(5)

Se ci fossero variabili non quantitative e non ordinali, potrei usare il metodo di pandas get_dummies()

In [ ]:
diamonds2=sns.load_dataset("diamonds")

In [ ]:
pd.get_dummies(data = diamonds2["color"], dtype=int).head(5)

In [ ]:
diamonds.info()

Eseguiamo il metodo describe

In [ ]:
diamonds.describe(include='all')

In [ ]:
diamonds[["x"]].boxplot()

In [ ]:
len(diamonds.query("x==0"))

sostituiamo gli zero nelle colonne x, y e z con dei null

In [ ]:
diamonds = diamonds.replace(to_replace={'x':{0:np.nan}, 
                                        'y':{0:np.nan},
                                        'z':{0:np.nan}
                             })

ora sono presenti dei null

In [ ]:
diamonds.info()

In [ ]:
features_training,features_test, target_training, target_test  = train_test_split(diamonds.drop("price",axis=1),
                                                                                   diamonds[["price"]],
                                                                                   test_size = 0.3,
                                                                                   shuffle = True,
                                                                                   random_state = 1)

Creiamo una pipeline standard con i passi
- simple imputer
- standard scaler
- regressione lineare

In [ ]:
basic_pipeline = Pipeline([('si', SimpleImputer()),
                           ('st', StandardScaler()),
                           ('rg', LinearRegression())
                          ])

Alleniamo la pipeline

In [ ]:
basic_pipeline.fit(features_training,target_training)

Vediamo le predizioni

In [ ]:
predizioni = basic_pipeline.predict(features_test)
predizioni

Analizziamo lo score

In [ ]:
basic_pipeline.score(features_test,target_test)

Ricomponiamo il Dataset completo

In [ ]:
diamonds_completo = features_test.copy()
diamonds_completo["prezzo_originale"]=target_test["price"]
diamonds_completo["prezzo_predetto"]=predizioni
diamonds_completo.head(10)

## Clustering

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
titanic=sns.load_dataset("titanic")

eliminiamo feature ridondanti e correlate

In [ ]:
titanic = titanic.drop(["alive","pclass","embarked","deck","adult_male","sex","alone"],axis=1)

In [ ]:
titanic.head()

Decodifichiamo le variabili non quantitative

In [ ]:
codifica_class = {"First":1, "Second":2, "Third":3}
titanic["class"] = titanic["class"].map(codifica_class).astype("int64")

In [ ]:
titanic_2 = pd.get_dummies(data=titanic, dtype=int)
titanic_2.head(5)

Valorizziamo i null

In [ ]:
imputer = SimpleImputer(strategy='mean')

In [ ]:
imputer.fit(titanic_2)

In [ ]:
titanic_2_imputed = pd.DataFrame(
    imputer.transform(titanic_2),
    columns=titanic_2.columns,
    index=titanic_2.index
)

Istanziamo l'algoritmo Kmeans di Scikit-learn

In [ ]:
km = KMeans (n_clusters = 2,
             n_init = 5,
             random_state = 0)

Eseguiamo il fit e predict su tutti i dati tranne survived

In [ ]:
cluster = km.fit_predict(titanic_2_imputed.drop("survived",axis=1))
cluster[:5]

Creiamo la colonna "cluster"

In [ ]:
titanic["cluster"] = cluster

In [ ]:
titanic.head(10)

In questo caso i due cluster non sembrano corrispondere con la suddivisione tra survived e non survived.

Tuttavia la suddivisione in clustered ha senso compiuto anche se non applicata ad un problema di classificazione